<a href="https://colab.research.google.com/github/anacasicande/Modelamiento/blob/main/Tarea2_Ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# Constantes
R = 0.08314  # bar·L/(mol·K), constante de los gases
T = 350  # K, temperatura
P = 9.4573  # bar, presión de vapor saturado

# Datos críticos del n-butano
T_c = 425.1  # K, temperatura crítica
P_c = 37.96  # bar, presión crítica

# Cálculo de los parámetros de Redlich-Kwong
a = 0.42748 * (R**2) * (T_c**2.5) / P_c
b = 0.08664 * R * T_c / P_c

# Definir la función Redlich-Kwong
def f(V_m):
    return (R * T) / (V_m - b) - (a / (V_m * (V_m + b) * np.sqrt(T))) - P

# Definir la derivada de la ecuación Redlich-Kwong
def f_prime(V_m):
    term1 = -(R * T) / (V_m - b)**2
    term2 = (a * (2 * V_m + b)) / (V_m**2 * (V_m + b)**2 * np.sqrt(T))
    return term1 - term2

# Método de Newton-Raphson
def newton_raphson(x0, tol=1e-6, max_iter=100):
    x = x0
    for i in range(max_iter):
        fx = f(x)
        fpx = f_prime(x)

        if abs(fpx) < 1e-8:
            print("La derivada es muy pequeña, no se puede continuar")
            return None

        # Fórmula de Newton-Raphson
        x_new = x - fx / fpx

        # Verificar la convergencia
        if abs(x_new - x) < tol:
            print(f"Converge en la iteración {i+1}")
            return x_new

        x = x_new

    print("No converge después del número máximo de iteraciones")
    return None

# Valores iniciales para la fase vapor y la fase líquida
V_vapor_initial = 0.2  # Volumen inicial de vapor saturado (en L/mol)
V_liquid_initial = 0.005  # Volumen inicial de líquido saturado (en L/mol)

# Llamar al método de Newton-Raphson para vapor saturado
V_vapor = newton_raphson(V_vapor_initial)
if V_vapor is not None:
    print(f"El volumen molar del vapor saturado es: {V_vapor:.6f} L/mol")

# Llamar al método de Newton-Raphson para líquido saturado
V_liquid = newton_raphson(V_liquid_initial)
if V_liquid is not None:
    print(f"El volumen molar del líquido saturado es: {V_liquid:.6f} L/mol")

Converge en la iteración 41
El volumen molar del vapor saturado es: 0.133269 L/mol
Converge en la iteración 3
El volumen molar del líquido saturado es: 0.000000 L/mol


In [1]:
import numpy as np

# Definir la función f(x) del problema, puede ser la ecuación de equilibrio o de estado
def f(x):
    # Ejemplo: f(x) = x^3 - 2x^2 + 4x - 8
    return x**3 - 2*x**2 + 4*x - 8

# Definir la derivada de la función f'(x)
def f_prime(x):
    # Derivada de la ecuación: f'(x) = 3x^2 - 4x + 4
    return 3*x**2 - 4*x + 4

# Implementación del método de Newton-Raphson
def newton_raphson(x0, tol=1e-6, max_iter=100):
    x = x0
    for i in range(max_iter):
        fx = f(x)
        fpx = f_prime(x)

        if abs(fpx) < 1e-8:
            print("La derivada es muy pequeña, no se puede continuar")
            return None

        # Fórmula de Newton-Raphson
        x_new = x - fx / fpx

        # Verificar la convergencia
        if abs(x_new - x) < tol:
            print(f"Converge en la iteración {i+1}")
            return x_new

        x = x_new

    print("No converge después del número máximo de iteraciones")
    return None

# Valor inicial de prueba
x0 = 2.0  # Elegir un valor inicial adecuado según el problema

# Llamar al método de Newton-Raphson
solucion = newton_raphson(x0)

if solucion is not None:
    print(f"La solución es: {solucion}")

Converge en la iteración 1
La solución es: 2.0
